In [1]:
import time
from collections import defaultdict

from Feynman.cloud import Google_drive
from Feynman.serialize import Pickle_serializer


ps = Pickle_serializer()
gd = Google_drive('../../../token.pickle')

[INFO    ] 2020-03-18 03:12:16 [pickle_serializer.py] [load:15] Pickle load : ../../../token.pickle
[INFO    ] 2020-03-18 03:12:16 [google_drive.py] [__init__:26] Google drive access...


## 유저 생성

In [2]:
gd.download(folder='demo_user_data',
           path='cache')

[INFO    ] 2020-03-18 03:13:17 [google_drive.py] [download:140] Connection reset by peer...
[INFO    ] 2020-03-18 03:15:18 [google_drive.py] [download:140] Connection reset by peer...
[INFO    ] 2020-03-18 03:17:18 [google_drive.py] [download:140] Connection reset by peer...
[INFO    ] 2020-03-18 03:18:19 [google_drive.py] [_download:120] demo_user.ps(1ht0fDDb0cDUxDHfiJVh_eVD3Hzwz0XVD) is the latest version...


In [3]:
demo_user = ps.load('cache/demo_user_data/demo_user.ps')

[INFO    ] 2020-03-18 03:18:19 [pickle_serializer.py] [load:15] Pickle load : cache/demo_user_data/demo_user.ps


In [4]:
demo_user.keys()

dict_keys(['item_count', 'user_count', 'cluster', 'traffic', 'timing', 'item_idx', 'user_idx', 'p_item_cluster', 'p_cluster_user', 'p_user'])

In [5]:
_timing = demo_user['timing']
_traffic = demo_user['traffic']
_user_count = demo_user['user_count']
_p_user = demo_user['p_user']

In [6]:
def make_user_list():
    user_num = _traffic/24/60/60*_timing
    user_num = np.random.poisson(user_num)
    u_idxs = np.random.choice(range(_user_count),user_num,p=_p_user[0])
    return {'user_id':list(map(int,u_idxs))}

In [7]:
u_list = make_user_list()

In [8]:


dic_msg = {}
dic_msg['type'] = 'user_list'
dic_msg['value'] = u_list
dic_msg['timestamp'] = time.ctime()
dic_msg['servive'] = 'demo_personal_reco_systemp'

In [9]:
dic_msg

{'type': 'user_list',
 'value': {'user_id': [39934,
   821337,
   92246,
   803002,
   897467,
   496192,
   643732,
   942236,
   180819,
   454341,
   782589,
   118032,
   620687,
   473116,
   274604,
   95529,
   846071,
   24490,
   658809,
   435269,
   101369,
   29092,
   649728,
   940683,
   139490,
   530482,
   906615,
   237854,
   471426,
   71325,
   445817,
   396661,
   518010,
   592038,
   197289,
   221342,
   506045,
   29395]},
 'timestamp': 'Wed Mar 18 03:18:21 2020',
 'servive': 'demo_personal_reco_systemp'}

In [10]:
import json

In [11]:
json.dumps(dic_msg)

'{"type": "user_list", "value": {"user_id": [39934, 821337, 92246, 803002, 897467, 496192, 643732, 942236, 180819, 454341, 782589, 118032, 620687, 473116, 274604, 95529, 846071, 24490, 658809, 435269, 101369, 29092, 649728, 940683, 139490, 530482, 906615, 237854, 471426, 71325, 445817, 396661, 518010, 592038, 197289, 221342, 506045, 29395]}, "timestamp": "Wed Mar 18 03:18:21 2020", "servive": "demo_personal_reco_systemp"}'

In [12]:
len(u_list.values())

1

In [13]:
len(u_list['user_id'])

38

## 추천 리스트 만들기

In [14]:
gd.download(folder='demo_reco',
           path='cache')

[INFO    ] 2020-03-18 03:18:23 [google_drive.py] [_download:120] reco_cluster_user.ps(1O1gVU9Ui1vSIfVK9FCO0LfngvKf2qKIg) is the latest version...
[INFO    ] 2020-03-18 03:18:23 [google_drive.py] [_download:120] reco_item_cluster.ps(1lg3OR0JSM29a0N6jid8etKM1sd77QNGA) is the latest version...


In [15]:
reco_item_cluster = ps.load('cache/demo_reco/reco_item_cluster.ps')
reco_cluster_user = ps.load('cache/demo_reco/reco_cluster_user.ps')

[INFO    ] 2020-03-18 03:18:24 [pickle_serializer.py] [load:15] Pickle load : cache/demo_reco/reco_item_cluster.ps
[INFO    ] 2020-03-18 03:18:24 [pickle_serializer.py] [load:15] Pickle load : cache/demo_reco/reco_cluster_user.ps


In [16]:
reco_item_cluster.keys()

dict_keys(['cluster', 'reco_p_item_cluster'])

In [17]:
_reco_p_item_cluster = reco_item_cluster['reco_p_item_cluster']

In [18]:
reco_cluster_user.keys()

dict_keys(['user_count', 'cluster', 'reco_p_cluster_user', 'reco_user_id_dic'])

In [19]:
_reco_user_id_dic = reco_cluster_user['reco_user_id_dic']
_cluster = reco_cluster_user['cluster']
_reco_p_cluster_user = reco_cluster_user['reco_p_cluster_user']

In [20]:
def make_reco(user_id):
    
    if user_id in _reco_user_id_dic:    
        cluster = _reco_p_cluster_user[_reco_user_id_dic[user_id]]
    else:
        cluster = [1./_cluster for _ in range(_cluster)]
    
    dic = defaultdict(float)
    for k in range(_cluster):
        for item, prob in _reco_p_item_cluster[k].items():
            dic[item]+= prob*cluster[k]
    return sorted(dic.items(), key = lambda x: -x[1])[:10]

In [21]:
a = make_reco(478117)

In [22]:
b = {123:a, 133:a}

In [23]:
b.keys()

dict_keys([123, 133])

In [24]:
b

{123: [(17982, 0.002276975676420655),
  (29835, 0.0017865985399487534),
  (39427, 0.0017187136211895887),
  (42886, 0.0016688283139483396),
  (75845, 0.0016553292809951534),
  (80938, 0.0016460698405256343),
  (14497, 0.001615838859243333),
  (41928, 0.0015686362589538412),
  (80471, 0.0015276446945529452),
  (91109, 0.0015160138472513109)],
 133: [(17982, 0.002276975676420655),
  (29835, 0.0017865985399487534),
  (39427, 0.0017187136211895887),
  (42886, 0.0016688283139483396),
  (75845, 0.0016553292809951534),
  (80938, 0.0016460698405256343),
  (14497, 0.001615838859243333),
  (41928, 0.0015686362589538412),
  (80471, 0.0015276446945529452),
  (91109, 0.0015160138472513109)]}

## 유저 선호도

In [25]:
user_ids = [i for i in range(10)]

In [26]:
result = []
for user_id in user_ids:
    stat= np.random.choice(['pass', 'choice', 'click'],p=[0.4, 0.3, 0.3])
    if stat == 'pass':
        continue
    elif stat == 'choice':
        pass
    else:
        pass

In [48]:
d = [[1,2],[3,4],[5,6]]

In [ ]:
e = list(zip(*d))

In [ ]:
list(zip(*e))

In [52]:
set(list(zip(*d))[0])

{1, 3, 5}

In [43]:
list(zip(*d.__iter__()))

[(1, 3, 5), (2, 4, 6)]

In [ ]:
def func(*a):
    print(*a)

In [ ]:
func(*e)

In [ ]:
q, w = list(zip(*d))

In [ ]:
q

In [44]:
{1:0.3, 2:0.4}

{1: 0.3, 2: 0.4}

In [45]:
import json

In [47]:
json.loads(json.dumps({1:0.3, 2:0.4}))

{'1': 0.3, '2': 0.4}

In [55]:
set(np.array([1,2]))

{1, 2}

In [56]:
a, b = 1, 2
